In [1]:
import sympy
from phasor.utilities.ipynb.displays import *
#from phasor.utilities.ipynb.ipy_sympy import *
import scipy.linalg


import numpy.testing as np_test
import declarative

from test_SVD import SVD_gen_check, gen_rand_unitary
from phasor.matrix import DAG_algorithm
from phasor.matrix import SRE_matrix_algorithms
from phasor.matrix import scisparse_algorithm

asavefig.org_subfolder = 'plots'

Populating the interactive namespace from numpy and matplotlib


In [2]:
from functools import reduce
import tempfile

from scipy.sparse.linalg import use_solver
#use_solver(useUmfpack = True)

    
def SVD_compare_error(
    N = 10,
    length = 10,
    solvers = {
        scisparse_algorithm : dict(), 
        DAG_algorithm : dict(METIS_fname = tempfile),
    },
):
    U = gen_rand_unitary(N = N, length = length, unit_density = .1,)
    V = gen_rand_unitary(N = N, length = length, unit_density = .1,)

    seq = dict()
    req = dict()
    edge_map = dict()
    S_diags = []
    for idx in range(N):
        s_diag = 10**(-5 + 10 * np.random.random(length))
        edge_map[idx, idx] = s_diag
        S_diags.append(s_diag)
        seq[idx] = set([idx])
        req[idx] = set([idx])
    S = seq, req, edge_map
    condition = reduce(np.maximum, S_diags) / reduce(np.minimum, S_diags)

    M = SRE_matrix_algorithms.matrix_mult_sre(
        SRE_matrix_algorithms.matrix_mult_sre(U, S), V
    )

    SRE_matrix_algorithms.check_sre(M)

    print("SPARSITY FRAC: ", SRE_matrix_algorithms.SRE_count_sparsity(M))

    inputs_set = set(range(N))
    outputs_set = set(range(N))

    def solve(solver, **kwargs):
        Mseq, Mreq, Medge_map = SRE_matrix_algorithms.copy_sre(M)
        print(solver)
        sbunch = solver.inverse_solve_inplace(
            seq = Mseq,
            req = Mreq,
            edge_map = Medge_map,
            inputs_set = inputs_set,
            outputs_set = outputs_set,
            verbose = False,
            negative = False,
            **kwargs
        )

        Minv = sbunch.seq, sbunch.req, sbunch.edge_map
        SRE_matrix_algorithms.check_sre(M)
        SRE_matrix_algorithms.check_sre(Minv)

        Meye = SRE_matrix_algorithms.matrix_mult_sre(M, Minv)
        #pprint(Meye)
        em1_ll = dict()
        em0_ll = dict()
        for (k_t, k_f), edge in Meye[2].items():
            if (k_t == k_f):
                em1_ll[k_t, k_f] = -np.log10(np.maximum(abs(edge - 1), 1e-30))
            else:
                em0_ll[k_t, k_f] = -np.log10(np.maximum(abs(edge), 1e-30))
        
        all_e0 = []
        all_e1 = []
        
        for k, e in em0_ll.items():
            all_e0.append(e)
        for k, e in em1_ll.items():
            all_e1.append(e)
            
        if all_e0:
            all_e0 = np.concatenate(all_e0)
        else:
            all_e0 = []
            
        if all_e1:
            all_e1 = np.concatenate(all_e1)
        else:
            all_e1 = []
        return declarative.Bunch(
            solver = solver,
            Meye = Meye,
            em1_ll = em1_ll,
            em0_ll = em0_ll,
            all0 = all_e0,
            all1 = all_e1,
        )
    sdict = dict()
    for solver, kwargs in solvers.items():
        sdict[solver] = solve(solver, **kwargs)
    sdict['condition'] = condition
    return sdict
    #pprint(em1_ll)
    #pprint(em0_ll)


In [3]:
r = SVD_compare_error(
    N = 100,
    length = 100,
)

{(0, 0): 0.1565505628788909,
 (0, 99): 0.4751535284088555,
 (1, 1): array([  3.12784601e-01,   3.41490426e-01,   3.72830728e-01,
          4.07047289e-01,   4.44404076e-01,   4.85189285e-01,
          5.29717558e-01,   5.78332417e-01,   6.31408906e-01,
          6.89356493e-01,   7.52622224e-01,   8.21694170e-01,
          8.97105198e-01,   9.79437077e-01,   1.06932497e+00,
          1.16746233e+00,   1.27460625e+00,   1.39158330e+00,
          1.51929594e+00,   1.65872941e+00,   1.81095939e+00,
          1.97716029e+00,   2.15861428e+00,   2.35672123e+00,
          2.57300945e+00,   2.80914754e+00,   3.06695721e+00,
          3.34842737e+00,   3.65572948e+00,   3.99123426e+00,
          4.35753000e+00,   4.75744256e+00,   5.19405711e+00,
          5.67074199e+00,   6.19117465e+00,   6.75937004e+00,
          7.37971160e+00,   8.05698504e+00,   8.79641529e+00,
          9.60370679e+00,   1.04850875e+01,   1.14473570e+01,
          1.24979388e+01,   1.36449378e+01,   1.48972028e+01,
   

          5.91554944e-01 +2.91058374e-04j,
          5.82014361e-01 +2.48163184e-04j,
          5.72473778e-01 +2.05267994e-04j,
          5.62933196e-01 +1.62372803e-04j,
          5.53392613e-01 +1.19477613e-04j,
          5.43852030e-01 +7.65824230e-05j,
          5.34311447e-01 +3.36872328e-05j,
          5.24770865e-01 -9.20795734e-06j,
          5.15230282e-01 -5.21031475e-05j,
          5.05689699e-01 -9.49983377e-05j,
          4.96149117e-01 -1.37893528e-04j,
          4.86608534e-01 -1.80788718e-04j,
          4.77067951e-01 -2.23683908e-04j,
          4.67527368e-01 -2.66579098e-04j,
          4.57986786e-01 -3.09474289e-04j,
          4.48446203e-01 -3.52369479e-04j,
          4.38905620e-01 -3.95264669e-04j,
          4.29365038e-01 -4.38159859e-04j,
          4.19824455e-01 -4.81055049e-04j,
          4.10283872e-01 -5.23950239e-04j,
          4.00743290e-01 -5.66845430e-04j,
          3.91202707e-01 -6.09740620e-04j,
          3.81662124e-01 -6.52635810e-04j,
          3

        -0.39029362-0.41710853j, -0.40274716-0.41510813j,
        -0.41520069-0.41310772j, -0.42765423-0.41110732j,
        -0.44010777-0.40910691j, -0.45256131-0.40710651j,
        -0.46501484-0.4051061j , -0.47746838-0.4031057j ,
        -0.48992192-0.40110529j, -0.50237546-0.39910489j,
        -0.51482900-0.39710448j, -0.52728253-0.39510408j,
        -0.53973607-0.39310367j, -0.55218961-0.39110327j,
        -0.56464315-0.38910287j, -0.57709668-0.38710246j]),
 (45,
  45): array([ 0.67436256,  0.66780949,  0.66125642,  0.65470335,  0.64815028,
         0.64159721,  0.63504414,  0.62849107,  0.621938  ,  0.61538493,
         0.60883186,  0.60227879,  0.59572572,  0.58917265,  0.58261958,
         0.57606651,  0.56951344,  0.56296037,  0.5564073 ,  0.54985423,
         0.54330116,  0.53674809,  0.53019502,  0.52364195,  0.51708888,
         0.51053581,  0.50398274,  0.49742967,  0.4908766 ,  0.48432353,
         0.47777046,  0.47121739,  0.46466432,  0.45811125,  0.45155818,
         0.

         2.51762163-1.71627944j,  2.55570976-1.77038641j,
         2.59415934-1.82588409j,  2.63296712-1.88280354j,
         2.67212952-1.94117637j,  2.71164266-2.00103486j,
         2.75150233-2.06241187j,  2.79170400-2.12534093j,
         2.83224276-2.18985621j,  2.87311338-2.25599252j,
         2.91431022-2.32378535j,  2.95582727-2.39327088j,
         2.99765813-2.46448596j,  3.03979598-2.53746815j,
         3.08223357-2.61225571j,  3.12496322-2.68888765j,
         3.16797680-2.76740369j,  3.21126569-2.84784429j,
         3.25482081-2.9302507j ,  3.29863257-3.01466491j]),
 (74,
  74): array([ 0.90311407,  0.90405003,  0.90498598,  0.90592194,  0.9068579 ,
         0.90779385,  0.90872981,  0.90966577,  0.91060173,  0.91153768,
         0.91247364,  0.9134096 ,  0.91434555,  0.91528151,  0.91621747,
         0.91715343,  0.91808938,  0.91902534,  0.9199613 ,  0.92089725,
         0.92183321,  0.92276917,  0.92370513,  0.92464108,  0.92557704,
         0.926513  ,  0.92744895,  0.9283

{(0,
  0): array([ 0.26107976,  0.26446549,  0.26785121,  0.27123694,  0.27462266,
         0.27800839,  0.28139411,  0.28477984,  0.28816556,  0.29155128,
         0.29493701,  0.29832273,  0.30170846,  0.30509418,  0.30847991,
         0.31186563,  0.31525136,  0.31863708,  0.3220228 ,  0.32540853,
         0.32879425,  0.33217998,  0.3355657 ,  0.33895143,  0.34233715,
         0.34572288,  0.3491086 ,  0.35249432,  0.35588005,  0.35926577,
         0.3626515 ,  0.36603722,  0.36942295,  0.37280867,  0.3761944 ,
         0.37958012,  0.38296584,  0.38635157,  0.38973729,  0.39312302,
         0.39650874,  0.39989447,  0.40328019,  0.40666592,  0.41005164,
         0.41343736,  0.41682309,  0.42020881,  0.42359454,  0.42698026,
         0.43036599,  0.43375171,  0.43713744,  0.44052316,  0.44390888,
         0.44729461,  0.45068033,  0.45406606,  0.45745178,  0.46083751,
         0.46422323,  0.46760896,  0.47099468,  0.4743804 ,  0.47776613,
         0.48115185,  0.48453758,  0.4879

         0.72797895,  0.7260197 ,  0.72406045,  0.7221012 ,  0.72014195,
         0.71818269,  0.71622344,  0.71426419,  0.71230494,  0.71034569,
         0.70838643,  0.70642718,  0.70446793,  0.70250868,  0.70054943,
         0.69859017,  0.69663092,  0.69467167,  0.69271242,  0.69075316,
         0.68879391,  0.68683466,  0.68487541,  0.68291616,  0.6809569 ,
         0.67899765,  0.6770384 ,  0.67507915,  0.6731199 ,  0.67116064,
         0.66920139,  0.66724214,  0.66528289,  0.66332364,  0.66136438,
         0.65940513,  0.65744588,  0.65548663,  0.65352738,  0.65156812,
         0.64960887,  0.64764962,  0.64569037,  0.64373112,  0.64177186,
         0.63981261,  0.63785336,  0.63589411,  0.63393486,  0.6319756 ,
         0.63001635,  0.6280571 ,  0.62609785,  0.6241386 ,  0.62217934,
         0.62022009,  0.61826084,  0.61630159,  0.61434234,  0.61238308,
         0.61042383,  0.60846458,  0.60650533,  0.60454608,  0.60258682,
         0.60062757,  0.59866832,  0.59670907,  0.5

          3.54392377e+01 -5.03495945e+00j,
          3.71085552e+01 -8.31538028e+00j,
          3.86003423e+01 -1.19300630e+01j,
          3.98759775e+01 -1.58838066e+01j,
          4.08941544e+01 -2.01784426e+01j,
          4.16109730e+01 -2.48124336e+01j,
          4.19800694e+01 -2.97804566e+01j,
          4.19527898e+01 -3.50729713e+01j,
          4.14784097e+01 -4.06757785e+01j,
          4.05044037e+01 -4.65695709e+01j,
          3.89767695e+01 -5.27294795e+01j,
          3.68404087e+01 -5.91246211e+01j,
          3.40395678e+01 -6.57176503e+01j,
          3.05183443e+01 -7.24643234e+01j,
          2.62212579e+01 -7.93130777e+01j,
          2.10938915e+01 -8.62046355e+01j,
          1.50836041e+01 -9.30716371e+01j,
          8.14031509e+00 -9.98383138e+01j,   2.17363810e-01 -1.06420206e+02j]),
 (41,
  41): array([ 0.04952586,  0.04280505,  0.03785747,  0.03471126,  0.0333843 ,
         0.03388414,  0.03620793,  0.04034247,  0.04626424,  0.05393958,
         0.06332483,  0.0743666

        -0.46789434-0.11310952j, -0.46053078-0.11149289j,
        -0.45316722-0.10987625j, -0.44580366-0.10825961j,
        -0.43844010-0.10664298j, -0.43107654-0.10502634j,
        -0.42371298-0.10340971j, -0.41634943-0.10179307j,
        -0.40898587-0.10017644j, -0.40162231-0.0985598j ,
        -0.39425875-0.09694317j, -0.38689519-0.09532653j,
        -0.37953163-0.0937099j , -0.37216807-0.09209326j,
        -0.36480451-0.09047662j, -0.35744095-0.08885999j,
        -0.35007739-0.08724335j, -0.34271383-0.08562672j,
        -0.33535027-0.08401008j, -0.32798671-0.08239345j,
        -0.32062315-0.08077681j, -0.31325959-0.07916018j,
        -0.30589604-0.07754354j, -0.29853248-0.0759269j ,
        -0.29116892-0.07431027j, -0.28380536-0.07269363j,
        -0.27644180-0.071077j  , -0.26907824-0.06946036j,
        -0.26171468-0.06784373j, -0.25435112-0.06622709j,
        -0.24698756-0.06461046j, -0.23962400-0.06299382j,
        -0.23226044-0.06137719j, -0.22489688-0.05976055j,
        -0.217

         0.23992664,  0.13519481,  0.06081119,  0.02322799,  0.02570527,
         0.06802813,  0.1465254 ,  0.25438805,  0.38225983,  0.51904886,
         0.65288975,  0.77217285,  0.86655127,  0.92783842,  0.95071812,
         0.93320572,  0.8768203 ,  0.78645285,  0.66994204,  0.53739428,
         0.40030707,  0.27057164,  0.15944155,  0.07655646,  0.02910601,
         0.02120616,  0.05354216,  0.12330911,  0.22445528,  0.34820701,
         0.4838298 ,  0.61955944,  0.74362243,  0.84525727,  0.91564793]),
 (90, 90): array([ 0.01487273-0.03453032j]),
 (91, 44): 1,
 (91, 91): array([ 0.57857038+0.38206993j,  0.57340041+0.37314807j,
         0.56823043+0.36422621j,  0.56306046+0.35530435j,
         0.55789048+0.3463825j ,  0.55272051+0.33746064j,
         0.54755054+0.32853878j,  0.54238056+0.31961692j,
         0.53721059+0.31069506j,  0.53204062+0.30177321j,
         0.52687064+0.29285135j,  0.52170067+0.28392949j,
         0.51653070+0.27500763j,  0.51136072+0.26608577j,
         0.5

SPARSITY FRAC:  {'Nedges': 438, 'Nnodes': 100, 'density_lin': 4.38, 'density_sq': 0.0438}
<module 'phasor.matrix.DAG_algorithm' from '/home/mcculler/local/home_sync/projects/phasor/phasor/matrix/DAG_algorithm.py'>
NUM EDGES:  53
[38, 36, 19, 33, 60, 23, 92, 37, 10, 91, 34, 90, 13, 94, 14, 16, 89, 53, 88, 87, 17, 62, 86, 54, 26, 64, 85, 31, 41, 93, 24, 68, 96, 40, 45, 47, 32, 98, 28, 97, 84, 12, 11, 66, 46, 21, 9, 95, 39, 8, 55, 83, 82, 81, 80, 44, 7, 65, 99, 58, 79, 22, 78, 4, 69, 29, 35, 6, 77, 59, 76, 75, 50, 74, 1, 18, 73, 51, 63, 27, 3, 15, 25, 72, 49, 71, 56, 52, 0, 70, 20, 2, 30, 57, 61, 48, 67, 43, 5, 42]
USING METIS
USING COL ALT
IMPROVED
USING COL ALT
USING COL ALT
IMPROVED
USING COL ALT
IMPROVED
USING COL ALT
IMPROVED
USING COL ALT
IMPROVED
USING COL ALT
IMPROVED
USING COL ALT
IMPROVED
USING COL ALT
IMPROVED
USING COL ALT
USING COL ALT
IMPROVED
USING COL ALT
USING COL ALT
IMPROVED
USING COL ALT
IMPROVED
USING COL ALT
IMPROVED
USING COL ALT
USING COL ALT
USING COL ALT
IMPROVED

In [4]:
b1 = r[DAG_algorithm]
b2 = r[scisparse_algorithm]

In [5]:
Meye1 = b1.Meye[2]
Meye2 = b2.Meye[2]

diag = []
offdiag = []
diag1 = []
offdiag1 = []
diag2 = []
offdiag2 = []
offdiag2x = []
for (k1, k2), e1 in Meye1.items():
    e2 = Meye2.get((k1, k2), np.zeros_like(e1))
    e_diff = e1 - e2
    if k1 == k2:
        diag.append(e_diff)
        diag1.append(e1)
        diag2.append(e2)
    else:
        offdiag.append(e_diff)
        offdiag1.append(e1)
        offdiag2.append(e2)
        offdiag2x.append(b2.em0_ll.get((k1, k2), np.zeros_like(e1)))
diag = np.concatenate(diag)
offdiag = np.concatenate(offdiag)
diag1 = np.concatenate(diag1)
offdiag1 = np.concatenate(offdiag1)
diag2 = np.concatenate(diag2)
offdiag2 = np.concatenate(offdiag2)
offdiag2x = np.concatenate(offdiag2x)

In [6]:
axB = mplfigB(Nrows=4)
Nbins = 30

_ = axB.ax0.hist(b1.all0, normed = 1, bins = Nbins)
_ = axB.ax0.hist(b1.all1, normed = 1, alpha = .5, bins = Nbins)
axB.ax0.set_yscale('log')
print(np.min(b1.all0), np.max(b1.all0))
print(np.min(b1.all1), np.max(b1.all1))
axB.ax0.set_xlim(0, 30)
axB.ax0.set_title('DAG_algorithm')

_ = axB.ax1.hist(b2.all0, normed = 1, bins = Nbins)
_ = axB.ax1.hist(b2.all1, normed = 1, alpha = .5, bins = Nbins)
print(np.min(b2.all0), np.max(b2.all0))
print(np.min(b2.all1), np.max(b2.all1))
axB.ax1.set_yscale('log')
axB.ax1.set_xlim(0, 30)
axB.ax1.set_title('Scisparse')

_ = axB.ax2.hist(-np.log10(np.maximum(abs(offdiag), 1e-30)), normed = 1, bins = Nbins)
_ = axB.ax2.hist(-np.log10(np.maximum(abs(diag), 1e-30)), normed = 1, alpha = .5, bins = Nbins)
axB.ax2.set_yscale('log')
axB.ax2.set_xlim(0, 30)
axB.ax2.set_title('unknown')

_ = axB.ax3.hist(np.log10(r['condition']))
axB.ax3.set_title('condition numbers')

axB.save('comparison')

1.43308195499 30.0
6.65986851679 30.0
0.849553239386 30.0
10.7311080462 30.0


![plots/comparison](plots/comparison.png?1504985748 "plots/comparison")
[pdf](plots/comparison.pdf),  [png](plots/comparison.png)

In [7]:
r['condition']

array([  5.07802530e+09,   6.68438335e+09,   6.56381255e+09,
         6.04197669e+09,   9.60050069e+09,   7.55191090e+09,
         6.56333215e+09,   4.37464533e+09,   7.31843034e+09,
         5.78122822e+09,   8.29979460e+09,   8.01520559e+09,
         7.68195012e+09,   6.16012199e+09,   8.47867375e+09,
         7.06684366e+09,   7.22194519e+09,   5.32202387e+09,
         8.23783695e+09,   6.15027043e+09,   8.95649460e+09,
         4.58742550e+09,   9.08633721e+09,   8.19970582e+09,
         6.61960298e+09,   7.11320970e+09,   7.53672537e+09,
         5.29564518e+09,   8.54194227e+09,   3.75224637e+09,
         9.11378842e+09,   7.04898349e+09,   5.40103015e+09,
         4.97483757e+09,   6.88031062e+09,   5.63091561e+09,
         4.88509835e+09,   6.41532031e+09,   7.99920055e+09,
         7.14722361e+09,   8.47140079e+09,   8.99326091e+09,
         7.81130171e+09,   8.36908136e+09,   8.78800272e+09,
         6.18042305e+09,   8.21976062e+09,   9.57482655e+09,
         9.01618714e+09,

In [8]:
idx_max = np.argmax(np.log10(abs(offdiag2)))
print(-np.log10(abs(offdiag2[idx_max])))
print(offdiag2x[idx_max])


0.849553239386
0.849553239386


/home/mcculler/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


In [9]:
idx_max = np.argmax(np.log10(abs(offdiag1)))
print(-np.log10(abs(offdiag1[idx_max])))

1.43308195499


/home/mcculler/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.
